# Tests Interpolation

In [ ]:
from interpol import Interpolation
import numpy as np
import meshplot as mp

Curva 2 puntos

In [ ]:
two = Interpolation( [[0,0,0], [1,1,1] ] )

p = mp.plot(np.array(list(two.evaluateFrom( i for i in np.linspace(0,1,100)) )), shading={'point_size':0.1}, return_plot=True)
p.add_points( np.array([[0,0,0], [1,1,1]]), shading={'point_size':0.4, 'point_color': 'blue'})

Curva 3 puntos

In [ ]:
three = Interpolation( [[0,0,0], [1,1,1], [0,0,2] ] )

p = mp.plot(np.array(list(three.evaluateFrom( i for i in np.linspace(0,1,100)) )), shading={'point_size':0.1}, return_plot=True)
p.add_points( np.array([[0,0,0], [1,1,1], [0,0,2]]), shading={'point_size':0.4, 'point_color': 'blue'})

# Tests submeshing

In [ ]:
from submesh import SkeletonMesh
import meshplot as mp
import matplotlib.cm as cm
import numpy as np

In [ ]:
skel = SkeletonMesh( 
        '../../data/humans/mesh/test_6890.off', 
        '../../data/humans/centerline/faust_6890.txt',
        '../../data/humans/centerline/faust_6890.polylines.txt' )

skel.submesh(pointsPerUnit=5)
skel.postprocess()

cmap = cm.get_cmap('flag')
p = None
for i, (joint, mesh, _) in enumerate(skel.getSubmeshes()):
    
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/skel.amountOfJoints))[0:3] )
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/skel.amountOfJoints))[0:3] )

In [ ]:
mp.plot( np.array(list(skel.getJoints())), shading={'point_size':0.1} )

In [ ]:
skel.centerAndNormalize()

cmap = cm.get_cmap('flag')
p2 = None
for i, (joint, mesh, _) in enumerate(skel.getSubmeshes()):
    
    if p2 is None:
        p2 = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/skel.amountOfJoints))[0:3] )
    else:
        p2.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/skel.amountOfJoints))[0:3] )

In [ ]:
p2.add_lines( np.array([[0,0,0], [0,0,0], [0,0,0] ]), np.array([[1,0,0], [0,1,0], [0,0,1] ]))

In [ ]:
skel.saveToJson( 'prueba.json')

# Test curvatura

### computar desde 0:

In [ ]:
import pymeshlab as pm
import meshplot as mp
import numpy as np
import open3d as o3d

ms = pm.MeshSet()
ms.load_new_mesh('../../data/humans/mesh/test_15070.off')

m = ms.current_mesh()

d = ms.apply_filter("compute_scalar_by_discrete_curvature_per_vertex", curvaturetype='Mean Curvature')
ms.compute_new_custom_scalar_attribute_per_vertex(name="v_curv", expr="q")

v_curv = m.vertex_custom_scalar_attribute_array('v_curv')

v_curv = np.clip( v_curv, a_min = -1 * float(d['90_percentile']), a_max = float(d['90_percentile']))


In [ ]:
mesh = o3d.io.read_triangle_mesh('../../data/humans/mesh/test_15070.off')
mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=v_curv)

### En skeleton mesh:

In [1]:
from submesh import SkeletonMesh
import meshplot as mp
import numpy as np

skel = SkeletonMesh( 
        '../../data/humans/mesh/test_6890.off', 
        '../../data/humans/centerline/faust_6890.txt',
        '../../data/humans/centerline/faust_6890.polylines.txt' )

skel.submesh(pointsPerUnit=5)
skel.postprocess()


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:

p = None
s = 0
for i, (joint, mesh, curv) in enumerate(list(skel.getSubmeshes())):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=curv )
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=curv )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0797183…